
## Project:  Allstate Claims Severity
#### Author:   Joshep Downs, James Peng, Megan Pera, Diana Rodenberger 
#### Purpose:  Create dummy variables for train and test data and save datasets to .csv files
#### Created:  12/06/2016

### Team name in Kaggle: UCB_207_1

## Link to Leaderboard
https://www.kaggle.com/c/allstate-claims-severity/leaderboard


In [1]:
%matplotlib inline
import unittest

# General libraries.
import re, os, sys
import numpy as np
import pandas as pd
import logging
from sklearn.pipeline import Pipeline
#from sklearn.feature_extraction import 
from sklearn import preprocessing

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


## Important note:

When transforming categorical variables into dummy variables, we get a different set of variables for the training data and the test data. The train data and the test data have different categories for some categorical variables; thus, when we tranform  categorical variables to dummy variables then a different set of dummy variables for the train and test data. 

The following code addresses this problem by adding dummy variables to the train data that only exist in the set data and viceversa. The newly added dummy variables in each set are set to 0.



### Load train data and test data

In [2]:
df_train = pd.read_csv('./data_in/train.csv')
df_train.shape

(188318, 132)

In [3]:
df_test = pd.read_csv('./data_in/test.csv')
df_test.shape

(125546, 131)

### Extract categorical variables from train and test data

 1) Extract data and load into dataframe
 
 2) Create dummy variables for categorical variables
 



In [4]:
#Extract categorical variables from AllState dataset
def GetColNamesByType(df, col_type):
    cat_cols=[]
    
    for c in df.columns:
        if c.find(col_type) >=0: # -1: substring not found, >=0, starting index
            cat_cols.append(c)
       
    return cat_cols

In [5]:
cat_cols_train=GetColNamesByType(df=df_train,col_type='cat')
#convert categorical variables into dummy variables for train data
df_dummy_vars_train=pd.get_dummies(df_train[cat_cols_train])

In [6]:
cat_cols_test=GetColNamesByType(df=df_train,col_type='cat')
#convert categorical variables into dummy variables for test data
df_dummy_vars_test=pd.get_dummies(df_test[cat_cols_test])

### Identify dummy variables in train data that do not exist in test data and viceversa. 

In [7]:
#get the list of dummy variables for each set.
dummy_cols_train=list(df_dummy_vars_train.columns)
dummy_cols_test=list(df_dummy_vars_test.columns)

#get list of dummy vars that exist only in training set
cols_train_only= np.setdiff1d(dummy_cols_train,dummy_cols_test)
print('Num of dummy vars in train set but not in test set:',len(cols_train_only) )
print('Example of 10 vars in train set but not in test set: ', cols_train_only[0:10])

#get list of dummy vars that exist only in test set
cols_test_only= np.setdiff1d(dummy_cols_test,dummy_cols_train)
print('Num of dummy vars in train set but not in test set: ', len(cols_test_only))
print('Example of 10 vars in train set but not in test set: ',cols_test_only[0:10])

Num of dummy vars in train set but not in test set: 74
Example of 10 vars in train set but not in test set:  ['cat101_N' 'cat101_U' 'cat102_H' 'cat102_J' 'cat105_R' 'cat105_S'
 'cat109_AG' 'cat109_AK' 'cat109_B' 'cat109_BF']
Num of dummy vars in train set but not in test set:  37
Example of 10 vars in train set but not in test set:  ['cat103_M' 'cat106_Q' 'cat109_AD' 'cat110_BH' 'cat110_CA' 'cat110_EN'
 'cat111_L' 'cat113_AA' 'cat113_R' 'cat116_A']


### Add missing dummy variables to train set and test set to create a complete set of variables for both train and test data

In [8]:
#add dummy variables to train set that only exist in test set
for column in cols_test_only:
    dummy_cols_train.append(column)
    #add the column at the beginning of the dataset
    df_dummy_vars_train.insert(0, column, 0, allow_duplicates=False)


In [9]:
#add dummy variables to test set that only exist in train set
for column in cols_train_only:
    dummy_cols_test.append(column)
    df_dummy_vars_test.insert(0, column, 0, allow_duplicates=False)

#### Sort dummy variables according to the order defined by AllState

AllState ordered categorical variables using the number first, and then the characters. AllState used a convention similar to Excel to sort the character part of the column name where 'Z' comes before 'AA'.

Steps to sort dummy variables: 

    1) Standardize names to meet the format 'catXXX_YY' where XXX is the original number of the categorical variable padded with zeros and YY are the original characters in the categorical variable padded with zeros.
    
    2) Create array of sorted indeces.
    
    3) Sort dummy variables using the index array. 




The following function transforms names of categorical variables to meet the format 'catXXX_YY' by adding zeros to numbers (XXX) with less than 3 digits and to names (YY) with less than one letter.
For example: 
    'cat2_A' --> 'cat002_0A'
    
Prior to this transformation, the string comparison 'cat20_AA' > 'cat101_C returns 'True'. After transformation, the string comparison 'cat020_AA' > cat101_0C' returns False.

In [10]:
### The following function transforms the number in the column to be three digits (with zero padding) and
### character part of the column (e.g. 'A', 'MB') to be two characters (by adding '0' for columns with one character) 
### with the purpose of sorting the columns in the same order provided by AllState.

### Example:  before transformation column 'cat20_AA' > 'cat101_C'
###           After transformation column 'cat020_AA' < cat101_0C' 

def RenameCatVars(columns):

    trans_columns=[]
    new_col=''
    for column in columns:
        new_col=column
        if column[:3]=='cat':
            delim_pos=int(np.core.defchararray.find(column,'_'))
            col_num=column[3:delim_pos]
            col_char=column[delim_pos+1:]
            
            #pad column number with zeros
            if len(col_num)==1:
                col_num='00'+ col_num
            elif len(col_num)==2:
                col_num='0'+ col_num

            #pad column char with '0'

            if len(col_char)==1:
                col_char='0'+ col_char
            
            new_col='cat'+ col_num + '_' + col_char
        trans_columns.append(new_col)
        
    return np.array(trans_columns)

In [11]:

def SortCatVariables(columns):
    
    #purpose: sort dummy variables in the order provided by AllState
    
    sorted_cols=[]
    
    #rename cat variables prior to sort them
    renamed_cols=RenameCatVars(columns)
    #create array with sorted indexes
    sorted_indx=np.argsort(renamed_cols)

    #create output array with original names sorted as defined by AllState
    for ii in range(len(sorted_indx)):
        sorted_cols.append(columns[sorted_indx[ii]])
    return sorted_cols

Sort complete set of dummy vars in train data and set data

In [12]:
dummy_cols_train=SortCatVariables(dummy_cols_train)
dummy_cols_test=SortCatVariables(dummy_cols_test)

Add missing dummy variables to train data and test data

In [13]:
df_dummy_vars_train=df_dummy_vars_train[dummy_cols_train]

In [14]:
df_dummy_vars_test=df_dummy_vars_test[dummy_cols_test]

### Create final X, y and Id sets for train and test data

In [15]:

cont_cols_test=GetColNamesByType(df=df_test,col_type='cont')

In [16]:
#get the list of continous variables
cont_cols_train=GetColNamesByType(df=df_train,col_type='cont')

#create dataset with predictors
X_train = pd.concat([df_dummy_vars_train, df_train[cont_cols_train]], axis=1)
#create dataset with dependent variable
y_train = np.log10(df_train.loss)
#create additioanl dataset with claim ids
id_train = df_train.id


In [17]:
#get the list of continous variables
cont_cols_test=GetColNamesByType(df=df_test,col_type='cont')

#create dataset with predictors
X_test = pd.concat([df_dummy_vars_test, df_test[cont_cols_test]], axis=1)

#create additioanl dataset with claim ids
id_test = df_test.id


In [18]:
#Training dataset should have one more variable (loss)
print(X_test.shape)
print(id_test.shape)
print(X_train.shape)
print(y_train.shape)
print(id_train.shape)

(125546, 1190)
(125546,)
(188318, 1190)
(188318,)
(188318,)


### Save final datasets to files

In [162]:
X_test.to_csv('./data_temp/X_dummies_test.csv', encoding='utf-8', index=False)

In [163]:
id_test.to_csv('./data_temp/id_test.csv', encoding='utf-8', index=False)

In [164]:
X_train.to_csv('./data_temp/X_dummies_train.csv', encoding='utf-8', index=False)

In [165]:
y_train.to_csv('./data_temp/y_train.csv', encoding='utf-8', index=False)

In [166]:
id_train.to_csv('./data_temp/id_train.csv', encoding='utf-8', index=False)